In [1]:
import os
import numpy as np
import cloudComPy as cc
import time
import matplotlib.pyplot as plt

from yaml import load
from yaml.loader import Loader

## Set up the processing parameters

In [2]:
cavename = "06_GrandFontannet"
SCAN = 0
SEGMENTS = np.arange(0, 15)
PROCESSING_FILEPATH = f"../../real_data/{cavename}/params.yaml"

p = load(open(PROCESSING_FILEPATH), Loader)
root = p["paths"]["root"]

SCAN_NAME = p['paths']['scanNames'][SCAN]

SIZES = []
CALC_TIMES = []

In [ ]:
for SEGMENT in SEGMENTS:

    SCAN_FP = os.path.normpath(os.path.join(root, "process", "cropped_clouds", f"{SCAN_NAME}_Cropped_{SEGMENT}.ply"))
    print(SCAN_FP)
    # define the file path, root should point to the home cave directory
    # load the point cloud to memory 
    cloud = cc.loadPointCloud(SCAN_FP)

    n, connected_components, residuals = cc.ExtractConnectedComponents([cloud], octreeLevel=10)
    print(f"there were {n} connected components")

    # we extract CC0, the largest component.
    sizes = [CC.size() for CC in connected_components]
    
    CC0 = connected_components[0]
    SIZES.append(CC0.size())
    
    # check that the largest is indeed CC0. 
    assert CC0.size() == max(sizes)

    start = time.time()
    CC0_norm = cc.computeNormals([CC0], model = cc.LOCAL_MODEL_TYPES.LS, defaultRadius= 0.08, orientNormalsMST=True, mstNeighbors=24)
    end = time.time()
    CALC_TIMES.append(end-start)
    print(f"time elapsed: {end-start}") # time in seconds.

    OUT_FP = os.path.normpath(os.path.join(root, "process", "cropped_clouds", f"{SCAN_NAME}_Cropped_{SEGMENT}_normals_reoriented.ply"))

    ret = cc.SavePointCloud(CC0, OUT_FP)

In [ ]:
fig, ax = plt.subplots()

ax.scatter(SIZES, CALC_TIMES)